In [3]:
from google.colab import drive
import os

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Path
path = "/content/drive/My Drive/"

In [ ]:
os.chdir("/content/drive/My Drive")


In [4]:
import os
import pandas as pd
ham_dir = "/content/drive/My Drive/NLP Project/input/ham-and-spam-dataset/ham" # Training dataset folder
spam_dir = "/content/drive/My Drive/NLP Project/input/ham-and-spam-dataset/spam" # Testing dataset folder
# Create a list containing all the words present in the training set emails
ham_emails = [os.path.join(ham_dir,f) for f in os.listdir(ham_dir)] # Gives us a path to all the messages stored in text files
spam_emails = [os.path.join(spam_dir,f) for f in os.listdir(spam_dir)]

**DATA PREPROCESSING**

In [5]:
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [6]:
path = '/content/drive/My Drive/NLP Project/input/ham-and-spam-dataset/hamnspam/'
mails = []
labels = []

for label in ['ham/', 'spam/']:
#     filenames = os.listdir(os.path.join(path, label))
    filenames = os.listdir(path + label)
    for file in filenames:
        f = open((path + label + file), 'r', encoding = 'latin-1')
        bolk = f.read()
        mails.append(bolk)
        labels.append(label)
        
df = pd.DataFrame({'emails': mails, 'labels': labels})

In [7]:
from sklearn.preprocessing import LabelEncoder
one_coder = LabelEncoder()
df['labels'] = one_coder.fit_transform(df['labels'])

In [8]:
df

,emails,labels
0,Return-Path: tim.one@comcast.net\nDelivery-Dat...,0
1,From spamassassin-talk-admin@lists.sourceforge...,0
2,From spamassassin-talk-admin@lists.sourceforge...,0
3,From spamassassin-commits-admin@lists.sourcefo...,0
4,From easmith@beatrice.rutgers.edu Thu Sep 26 ...,0
...,...,...
3047,From 2002biz2biz2513@Flashmail.com Mon Oct 7...,1
3048,From biz2biz2446@Flashmail.com Mon Oct 7 22:...,1
3049,From cna@insiq.us Tue Oct 8 00:10:39 2002\nR...,1
3050,From OWNER-NOLIST-SGODAILY*JM**NETNOTEINC*-COM...,1


In [9]:
df = df.drop_duplicates()

In [15]:
def remove_special_char(text):
    new_text = ""
    for i in text.split():
        if i.isalpha():
            new_text = new_text + " " + i
        
    return new_text

In [16]:
df['emails'] = df['emails'].apply(lambda x:x.lower())
df['emails'] = df['emails'].apply(lambda x:x.replace('\n', " "))
df['emails'] = df['emails'].apply(lambda x:x.replace('\t', " "))
df['emails'] = df['emails'].apply(remove_special_char)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [17]:
df

,emails,labels
0,fri sep sep test any thought to wrapping up y...,0
1,from tue sep from localhost by with esmtp id ...,0
2,from thu aug from localhost by with esmtp id ...,0
3,from thu aug from localhost by with esmtp id ...,0
4,from thu sep from localhost by with esmtp id ...,0
...,...,...
3047,from mon oct from localhost by with esmtp id ...,1
3048,from mon oct from localhost by with esmtp id ...,1
3049,from tue oct from localhost by with esmtp id ...,1
3050,from thu sep from localhost by with esmtp id ...,1


In [28]:
X = cv.fit_transform(df['emails']).toarray()

In [20]:
X.shape

(3047, 2500)

In [21]:
y = df['labels'].values

In [22]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [72]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test, = train_test_split(X,y,test_size=0.2,random_state=42)

In [73]:
from sklearn.metrics import accuracy_score

In [2]:
pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 14.5 MB/s 
     |████████████████████████████████| 462 kB 52.5 MB/s 


In [31]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [32]:
df['labels'].value_counts()

0    2550
1     497
Name: labels, dtype: int64

In [33]:
497/2550

0.19490196078431374

**19% spam emails, 81% ham emails: This indicates class imbalance**




In [34]:
df_spam = df[df['labels']==1]
df_spam.shape

(497, 2)

In [35]:
df_ham = df[df['labels']==0]
df_ham.shape

(2550, 2)

In [36]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(497, 2)

In [37]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(994, 2)

In [38]:
df_balanced['labels'].value_counts()

0    497
1    497
Name: labels, dtype: int64

In [39]:
df_balanced['spam']=df_balanced['labels'].apply(lambda x: 1 if x==1 else 0)
df_balanced.sample(5)

,emails,labels,spam
1989,from thu sep from localhost by with esmtp id ...,0,0
2614,from fri aug from localhost by with esmtp id ...,1,1
878,from sun oct from localhost by with esmtp id ...,0,0
2710,from mon sep from localhost by with esmtp id ...,1,1
239,from tue sep from localhost by with esmtp id ...,0,0


Split the dataset into train and test

In [58]:
from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test, = train_test_split(X,y,test_size=0.2,random_state=42)

X_train, X_test, y_train, y_test = train_test_split(df['emails'],df['labels'])

In [59]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [60]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [61]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [62]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [64]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
72/72 [==============================] - 988s 14s/step - loss: 0.4228 - accuracy: 0.8363 - precision: 0.3333 - recall: 0.0081
Epoch 2/10
72/72 [==============================] - 1001s 14s/step - loss: 0.3812 - accuracy: 0.8416 - precision: 0.7143 - recall: 0.0404
Epoch 3/10
72/72 [==============================] - 999s 14s/step - loss: 0.3521 - accuracy: 0.8429 - precision: 0.7000 - recall: 0.0566
Epoch 4/10
72/72 [==============================] - 1003s 14s/step - loss: 0.3341 - accuracy: 0.8486 - precision: 0.7778 - recall: 0.0943
Epoch 5/10
72/72 [==============================] - 1003s 14s/step - loss: 0.3210 - accuracy: 0.8617 - precision: 0.8161 - recall: 0.1914
Epoch 6/10
72/72 [==============================] - 1001s 14s/step - loss: 0.3012 - accuracy: 0.8691 - precision: 0.8913 - recall: 0.2210
Epoch 7/10
72/72 [==============================] - 1000s 14s/step - loss: 0.2904 - accuracy: 0.8801 - precision: 0.8702 - recall: 0.3073
Epoch 8/10
72/72 [==================

Evaluate the model

In [65]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [66]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Make Predictions

In [67]:
sample_dataset = [
    'You can win alot of money, register in the link below',
    'You have an iphone 10, spin the image below to claim your prize and it willl be delivered in your door step',
    'You have an offer, the company will give you 50% off in every item purchased.',
    'Hey Bravin, dont be late for the meeting tomorrow',
    "See you monday, we have alot to talk about the future of this company ."
]
model.predict(sample_dataset)

array([[0.80037606],
       [0.63915753],
       [0.8394575 ],
       [0.30569944],
       [0.27604842]], dtype=float32)

Predicted value below 5 are considered as ham and above 5 are considered as spam

In [68]:
from sklearn.metrics import accuracy_score
print("Accuracy of BERT is:",accuracy_score(y_test,y_predicted))


Accuracy of BERT is: 0.910761154855643


Naive bayes

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt # show graph
import re

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional

import os
import email
from bs4 import BeautifulSoup

In [7]:
path = '/content/drive/My Drive/NLP Project/input/ham-and-spam-dataset/hamnspam/'

topics = []
contents = []
labels = []
num_mails = []
content_types = []

count = 0
error = 0
wrongContentType = 0
for label in ['ham', 'spam']:
    filenames = os.listdir(path + label + '/')
    for filename in filenames:
        count +=1
        labels.append(1 if label == 'spam' else 0)
        with open(os.path.join(path + label + '/', filename), 'rb') as file:
            email_file = email.parser.BytesParser(policy=email.policy.default).parse(file)
            payload = email_file.get_payload()
            num_mails.append(1 if not isinstance(payload, list) else len(payload))
            try:
                if email_file.get_content_type() == 'text/plain':
                    contents.append(email_file.get_content())
                elif email_file.get_content_type() == 'text/html':
                    contents.append(BeautifulSoup(email_file.get_content()).body.text)
                else:
                    wrongContentType += 1
                    contents.append('')
                topics.append(email_file['Subject'])
                content_types.append(email_file.get_content_type())
            except LookupError:
                error +=1
                topics.append(None)
                contents.append('')
                content_types.append('')
                pass
        
inputDF = pd.DataFrame({'topic': topics, 'content': contents, 'label': labels, 'num_mails_in_flow': num_mails, 'content_type': content_types})
print('# processed message: %d', count)
print('# failed-parsed message: %d', error)
print('# non-text message: %d', wrongContentType)

# processed message: %d 3052
# failed-parsed message: %d 3
# non-text message: %d 196


In [8]:
inputDF.describe()

,label,num_mails_in_flow
count,3052.000000,3052.000000
mean,0.164155,1.050131
std,0.370477,0.224178
min,0.000000,1.000000
25%,0.000000,1.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,1.000000,3.000000


In [9]:
inputDF.head(5)


,topic,content,label,num_mails_in_flow,content_type
0,[Spambayes] test sets?,[Skip Montanaro]\n> Any thought to wrapping up...,0,1,text/plain
1,"[SAtalk] ""Broken Pipe"" on initial test",Hi\n\nI've just installed SpamAssassin and rel...,0,1,text/plain
2,Re: [SAtalk] O.T. Habeus -- Why?,"\nOn Wednesday, August 28, 2002, at 01:50 PM,...",0,1,text/plain
3,"[SACVS] CVS: spamassassin configure,1.1.2.1,1....",Update of /cvsroot/spamassassin/spamassassin\n...,0,1,text/plain
4,Re: GA Development (was Re: [SAdev] [Bug 1030]...,"On Sep 25, 7:35pm, Daniel Quinlan wrote:\n> A...",0,1,text/plain


In [10]:
inputDF.loc[inputDF['content_type'] == 'text/html'].head(5)


,topic,content,label,num_mails_in_flow,content_type
2558,Conference calls/best quality/$.18 per minute!,\n\n\n\n\n\nConferencing Made Easy\nOnly 18 Ce...,1,1,text/html
2560,Let me know what you think!32482,\n\n__________________________________________...,1,1,text/html
2562,Mothers you want to fuck,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,1,text/html
2569,Incredible Pictures!!!!!!!!,\nDo you like Sexy Animals doing the wild thin...,1,1,text/html
2572,- Custom Websites for $399 Complete! (1709D@5),"\n\n\n\n\n\nBeautiful, 100% Custom \n Web...",1,1,text/html


In [11]:
# It seems that an email of 'text/html' is likely to be a spam.


inputDF.loc[inputDF['content_type'].str.startswith('multipart')].head(5)


,topic,content,label,num_mails_in_flow,content_type
9,Re: [Razor-users] Razor with sendmail,,0,2,multipart/signed
17,Re: [SAtalk] Getting yourself removed from spa...,,0,2,multipart/signed
49,Re: [SAtalk] O.T. Habeus -- Why?,,0,2,multipart/signed
61,Re: [SAtalk] Still confused about spamd/c,,0,2,multipart/signed
95,[SAtalk] spamc and DCC,,0,2,multipart/alternative


In [12]:
inputDF['topic_content'] = inputDF.topic.astype('U') + " " + inputDF.content
inputDF.topic_content

0       [Spambayes] test sets? [Skip Montanaro]\n> Any...
1       [SAtalk] "Broken Pipe" on initial test Hi\n\nI...
2       Re: [SAtalk] O.T. Habeus -- Why? \nOn Wednesda...
3       [SACVS] CVS: spamassassin configure,1.1.2.1,1....
4       Re: GA Development (was Re: [SAdev] [Bug 1030]...
                              ...                        
3047    FWD:Direct marketing is working               ...
3048    See your Company sales sky rocket.            ...
3049                               Hit the Road with CNA 
3050    MAKE MONEY GIVING AWAY FREE STUFF! \n\n\n\n\n\...
3051    Faeries UNCOMMON EXOTIC PLEASURE BOTANICALS!\n...
Name: topic_content, Length: 3052, dtype: object

In [13]:
y = inputDF.label
X = inputDF.topic_content
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y, shuffle=True)
X_train.shape, X_test.shape

((2044,), (1008,))

In [14]:
def reportTest(y_pred, y_test):
    print("Classification report:")
    print(classification_report(y_test, y_pred))
    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("The accuracy is {}".format(accuracy_score(y_test, y_pred))) 
    print("The precision is {}".format(precision_score(y_test, y_pred))) 
    print("The recall is {}".format(recall_score(y_test, y_pred))) 
    print("The F1-Score is {}".format(f1_score(y_test, y_pred))) 
    print("The AUC is {} ".format(roc_auc_score(y_test, y_pred)))

In [15]:
def get_strategy():
    try:
        tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running on TPU ', tpu_cluster_resolver.cluster_spec().as_dict()['worker'])
        tf.config.experimental_connect_to_cluster(tpu_cluster_resolver)
        tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver)
        strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)
    except ValueError as e:
        print(e)
        print('No TPU detected')
        tpu = None
        strategy = tf.distribute.get_strategy()
    return strategy

strategy = get_strategy()

Running on TPU  ['10.71.242.82:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.71.242.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.71.242.82:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


MultinomialNB

In [16]:
# cv = CountVectorizer(stop_words = {'english'}, ngram_range=(1, 3))
cv = CountVectorizer(max_features=2500,stop_words="english")
feature_word_train = cv.fit_transform(X_train).toarray()
feature_word_test = cv.transform(X_test).toarray()

In [17]:
mnb = MultinomialNB()

def checkTest(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    reportTest(y_pred, y_test)

for clf, clf_name in [(mnb, 'MultinomialNB')]:
    print("Training for " + clf_name)
    clf.fit(feature_word_train, y_train)
    checkTest(clf, feature_word_test, y_test)
    print("--------------------------------------------------------------\n")

Training for MultinomialNB
Classification report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       843
           1       0.96      0.80      0.87       165

    accuracy                           0.96      1008
   macro avg       0.96      0.90      0.93      1008
weighted avg       0.96      0.96      0.96      1008

Confusion matrix:
[[838   5]
 [ 33 132]]
The accuracy is 0.9623015873015873
The precision is 0.9635036496350365
The recall is 0.8
The F1-Score is 0.8741721854304637
The AUC is 0.8970344009489917 
--------------------------------------------------------------



In [33]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB,ComplementNB

gnb = GaussianNB()

def checkTest(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    reportTest(y_pred, y_test)

for clf, clf_name in [(gnb, 'GaussianNB')]:
    print("Training for " + clf_name)
    clf.fit(feature_word_train, y_train)
    checkTest(clf, feature_word_test, y_test)
    print("--------------------------------------------------------------\n")

Training for GaussianNB
Classification report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       843
           1       0.87      0.82      0.84       165

    accuracy                           0.95      1008
   macro avg       0.92      0.90      0.91      1008
weighted avg       0.95      0.95      0.95      1008

Confusion matrix:
[[823  20]
 [ 30 135]]
The accuracy is 0.9503968253968254
The precision is 0.8709677419354839
The recall is 0.8181818181818182
The F1-Score is 0.84375
The AUC is 0.897228512886876 
--------------------------------------------------------------



In [34]:
bnb = BernoulliNB()

def checkTest(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    reportTest(y_pred, y_test)

for clf, clf_name in [(bnb, 'BernoulliNB')]:
    print("Training for " + clf_name)
    clf.fit(feature_word_train, y_train)
    checkTest(clf, feature_word_test, y_test)
    print("--------------------------------------------------------------\n")

Training for BernoulliNB
Classification report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       843
           1       0.88      0.64      0.74       165

    accuracy                           0.93      1008
   macro avg       0.90      0.81      0.85      1008
weighted avg       0.92      0.93      0.92      1008

Confusion matrix:
[[828  15]
 [ 59 106]]
The accuracy is 0.9265873015873016
The precision is 0.8760330578512396
The recall is 0.6424242424242425
The F1-Score is 0.7412587412587414
The AUC is 0.8123153240590963 
--------------------------------------------------------------



In [35]:
cnb = ComplementNB()

def checkTest(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    reportTest(y_pred, y_test)

for clf, clf_name in [(cnb, 'ComplementNB')]:
    print("Training for " + clf_name)
    clf.fit(feature_word_train, y_train)
    checkTest(clf, feature_word_test, y_test)
    print("--------------------------------------------------------------\n")

Training for ComplementNB
Classification report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       843
           1       0.96      0.87      0.91       165

    accuracy                           0.97      1008
   macro avg       0.97      0.93      0.95      1008
weighted avg       0.97      0.97      0.97      1008

Confusion matrix:
[[837   6]
 [ 21 144]]
The accuracy is 0.9732142857142857
The precision is 0.96
The recall is 0.8727272727272727
The F1-Score is 0.9142857142857144
The AUC is 0.9328049175024263 
--------------------------------------------------------------



LSTM model

In [18]:
num_words = 8196

tokenizer = Tokenizer(num_words = num_words - 1, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
seq_train_tokenized = tokenizer.texts_to_sequences(X_train)
seq_test_tokenized = tokenizer.texts_to_sequences(X_test)

In [19]:
max_content_length = 512

trained_word_embedding = pad_sequences(seq_train_tokenized, max_content_length)
test_word_embedding = pad_sequences(seq_test_tokenized, max_content_length)

In [21]:
tf.keras.backend.clear_session()

with strategy.scope():
    lstmModel = Sequential([
        Embedding(num_words, 256, input_length=max_content_length),
        Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
        Dropout(0.5),
        Bidirectional(tf.keras.layers.LSTM(64)),
        Dense(16, activation="relu"),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])

    lstmModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                      metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
    # lstmModel.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    lstmModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 256)          2098176   
                                                                 
 bidirectional (Bidirectiona  (None, 512, 256)         394240    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 512, 256)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0

In [22]:
X_new = np.array(trained_word_embedding)
y_new = np.array(y_train)

lstmModel.fit(X_new, y_new, epochs=40, batch_size=128)


Epoch 1/40


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 23s 407ms/step - loss: 0.6889 - accuracy: 0.5352
Epoch 2/40
16/16 [==============================] - 7s 406ms/step - loss: 0.6554 - accuracy: 0.7402
Epoch 3/40
16/16 [==============================] - 6s 406ms/step - loss: 0.6242 - accuracy: 0.7916
Epoch 4/40
16/16 [==============================] - 6s 406ms/step - loss: 0.5902 - accuracy: 0.8151
Epoch 5/40
16/16 [==============================] - 6s 406ms/step - loss: 0.5414 - accuracy: 0.8273
Epoch 6/40
16/16 [==============================] - 6s 405ms/step - loss: 0.5182 - accuracy: 0.8322
Epoch 7/40
16/16 [==============================] - 7s 406ms/step - loss: 0.4890 - accuracy: 0.8312
Epoch 8/40
16/16 [==============================] - 6s 405ms/step - loss: 0.4746 - accuracy: 0.8312
Epoch 9/40
16/16 [==============================] - 6s 405ms/step - loss: 0.4629 - accuracy: 0.8288
Epoch 10/40
16/16 [==============================] - 6s 405ms/step - loss: 0.4462 - accuracy: 0.8346
Epoch 11/

In [23]:
y_pred = np.argmax(lstmModel.predict(np.array(test_word_embedding)), axis=-1)
reportTest(y_pred, np.array(y_test))

Classification report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       843
           1       0.94      0.87      0.91       165

    accuracy                           0.97      1008
   macro avg       0.96      0.93      0.94      1008
weighted avg       0.97      0.97      0.97      1008

Confusion matrix:
[[834   9]
 [ 21 144]]
The accuracy is 0.9702380952380952
The precision is 0.9411764705882353
The recall is 0.8727272727272727
The F1-Score is 0.9056603773584905
The AUC is 0.9310255580718213 


Bert Model

In [24]:
!pip install -q transformers

# Loading the BERT Classifier and Tokenizer along with Input module
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputFeatures

     |████████████████████████████████| 4.0 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 6.6 MB 11.9 MB/s 
     |████████████████████████████████| 77 kB 3.6 MB/s 


In [25]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [26]:
features_train = tokenizer.batch_encode_plus(X_train, padding='max_length', truncation=True,
                                             max_length=512,return_tensors='tf',add_special_tokens=True,
                                             return_token_type_ids=True, return_attention_mask=True)

features_test = tokenizer.batch_encode_plus(X_test, padding='max_length', truncation=True,
                                            max_length=512,return_tensors='tf',add_special_tokens=True,
                                            return_token_type_ids=True, return_attention_mask=True)

In [27]:
tf.keras.backend.clear_session()

with strategy.scope():
    bertModel = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
    bertModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                      metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
#     bertModel.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    bertModel.summary()

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [28]:
bertModel.fit(x=features_train.data, y=y_train, epochs=10, batch_size=128, verbose=1)

Epoch 1/10
16/16 [==============================] - 135s 457ms/step - loss: 0.4166 - accuracy: 0.8278
Epoch 2/10
16/16 [==============================] - 7s 457ms/step - loss: 0.1624 - accuracy: 0.9251
Epoch 3/10
16/16 [==============================] - 7s 457ms/step - loss: 0.0509 - accuracy: 0.9883
Epoch 4/10
16/16 [==============================] - 7s 457ms/step - loss: 0.0151 - accuracy: 0.9961
Epoch 5/10
16/16 [==============================] - 7s 459ms/step - loss: 0.0060 - accuracy: 0.9985
Epoch 6/10
16/16 [==============================] - 7s 457ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 7/10
16/16 [==============================] - 7s 457ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 8/10
16/16 [==============================] - 7s 458ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 9/10
16/16 [==============================] - 7s 458ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 10/10
16/16 [==============================] - 7s 460ms/step - loss: 8.6399e-04 - accuracy: 

In [29]:
bert_outputs = bertModel(features_test.data)
bert_outputs

TFSequenceClassifierOutput([('logits',
                             <tf.Tensor: shape=(1008, 2), dtype=float32, numpy=
                             array([[ 3.6443212, -3.9770546],
                                    [-3.5374353,  3.7646701],
                                    [ 3.3564913, -3.7933304],
                                    ...,
                                    [ 3.3798435, -3.443226 ],
                                    [ 3.7145722, -3.9403367],
                                    [-3.502055 ,  3.787052 ]], dtype=float32)>)])

In [30]:
softmax_outputs = tf.nn.softmax(bert_outputs[0], axis=-1)
softmax_outputs

<tf.Tensor: shape=(1008, 2), dtype=float32, numpy=
array([[9.9951041e-01, 4.8962748e-04],
       [6.7366386e-04, 9.9932635e-01],
       [9.9921560e-01, 7.8438822e-04],
       ...,
       [9.9891281e-01, 1.0871917e-03],
       [9.9952650e-01, 4.7348859e-04],
       [6.8247167e-04, 9.9931753e-01]], dtype=float32)>

In [31]:
y_pred = np.argmax(softmax_outputs.numpy(), axis=1)
reportTest(y_pred, np.array(y_test))

Classification report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       843
           1       0.99      0.96      0.98       165

    accuracy                           0.99      1008
   macro avg       0.99      0.98      0.99      1008
weighted avg       0.99      0.99      0.99      1008

Confusion matrix:
[[842   1]
 [  7 158]]
The accuracy is 0.9920634920634921
The precision is 0.9937106918238994
The recall is 0.9575757575757575
The F1-Score is 0.9753086419753086
The AUC is 0.9781947589776772 
